<a href="https://colab.research.google.com/github/zoomer-k/pdf-diff/blob/main/pdf_diff_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 回路図差分チェックツール  
PDFの回路図を画像に変換して、差分をチェックします。

比較する回路図を以下の名前でフォルダーに追加してください。

new.pdf  
old.pdf

保存先　→（/content)フォルダー

追加後　ctrl + F9　もしくは、ランタイム→すべてのセルを実行を押してください。

比較結果が以下のファイル名で保存されます。

result.pdf



# 新しいセクション

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 変更履歴

## 1.02
* poppler-utilsがインストールできない問題に対応

## 1.01
* 生成されるresult.pdfの並びを番号順にソートされるように修正




斜体テキスト# 新しいセクション

In [ ]:
#https://qiita.com/k12u/items/2f9f04f94f0ce8daea67
!sudo apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [ ]:
!sudo apt install poppler-utils
#日本語フォントにはdataも必要
!sudo apt install poppler-data

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (355 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend

In [ ]:
!pip install pdf2image
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pdf2image
import os

In [ ]:
# フォルダ作成
!mkdir -p /content/result_img
!mkdir -p /content/target_img
!mkdir -p /content/source_img

In [ ]:
!ls

drive  new.pdf	old.pdf  result_img  sample_data  source_img  target_img


In [ ]:

#カレントディレクトリ内のファイルリストを取得
files = os.listdir('.')
for file in files:
    #PDFファイルだったら
    if 'new.pdf' in file:
        #PDFファイルを画像ファイルに変換
        images = pdf2image.convert_from_path(file, grayscale=True, size=4000)
        fname = os.path.splitext(file)[0]
        #ページごとに保存
        for index, image in enumerate(images):
            #newとoldで同じファイル名にする
            image.save("target_img/" + str(index+1) + '.png')
            #image.save("target_img/"+fname +"-"+ str(index+1) + '.png')
    if 'old.pdf' in file:
        #PDFファイルを画像ファイルに変換
        images = pdf2image.convert_from_path(file, grayscale=True, size=4000)
        fname = os.path.splitext(file)[0]
        #ページごとに保存
        for index, image in enumerate(images):
            image.save("source_img/" + str(index+1) + '.png')
            #image.save("source_img/"+fname +"-"+ str(index+1) + '.png')

In [ ]:
import pathlib
import cv2
import numpy as np

In [ ]:
source_dir = pathlib.Path('/content/source_img')
source_files = source_dir.glob('*.*')
target_dir = pathlib.Path('/content/target_img')
result_dir = pathlib.Path('/content/result_img')
log_file = result_dir / pathlib.Path('result.log')
#
kernel = np.ones((7, 7), np.uint8) 
#kernel = np.ones((3, 3), np.uint8)

In [ ]:
fs = open(log_file, mode='w')
for source_file in source_files:
    source_img = cv2.imread(str(source_file))
    target_file = target_dir / source_file.name
    target_img = cv2.imread(str(target_file))
    #print(target_img)
    if target_img is None:
        #fs.write(target_file + '...skipped.\n')
        continue
    #最大幅と最大高さの取得
    max_hight = max(source_img.shape[0], target_img.shape[0])
    max_width = max(source_img.shape[1], target_img.shape[1])

    temp_img = source_img
    source_img = np.zeros((max_hight, max_width, 3), dtype=np.uint8)
    source_img[0:temp_img.shape[0], 0:temp_img.shape[1]] = temp_img

    temp_img = target_img
    target_img = np.zeros((max_hight, max_width, 3), dtype=np.uint8)
    target_img[0:temp_img.shape[0], 0:temp_img.shape[1]] = temp_img
    # アルファブレンドで2枚を合成
    # https://note.nkmk.me/python-opencv-numpy-alpha-blend-mask/
    result_img = cv2.addWeighted(source_img, 0.5, target_img, 0.5, 0)
    #RGB⇔BGR変換
    #https://note.nkmk.me/python-opencv-bgr-rgb-cvtcolor/
    source_img = cv2.cvtColor(source_img, cv2.COLOR_BGR2GRAY)
    target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2GRAY)
    # 差の絶対値をもとめる。
    img = cv2.absdiff(source_img, target_img)
    # threshhold(src, thresh, maxval, type)
    # threshより大きな値がmaxvalにそれ以外が0に置き換わる。
    rtn, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    # 収縮→差分消えてしまう。
    #img = cv2.erode(img,kernel,iterations = 1)
    # 膨張
    img = cv2.dilate(img,kernel,iterations = 1)
    # クロージング →あまり意味ない？
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    # モルフォロジー勾配→輪郭抽出。
    # http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    # 輪郭を抽出する。
    # http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_contours/py_contours_begin/py_contours_begin.html
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
    result_img = cv2.drawContours(result_img, contours, -1, (0, 0, 255),2)
    score = 0
    for contour in contours:
        score += cv2.contourArea(contour)
    score /= max_hight * max_width
    fs.write(target_file.name + ', ' + str(score) + '\n')
    diff_file = result_dir / source_file.name
    cv2.imwrite(str(diff_file), result_img)
fs.close()

In [ ]:
!pip install img2pdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 2.3 MB 38.6 MB/s 
  Created wheel for img2pdf: filename=img2pdf-0.4.4-py3-none-any.whl size=44881 sha256=abb8aa6aeb8e75f5b13238ee402ec310ae7d9bf67e00709978bbc37f789eba58
  Stored in directory: /root/.cache/pip/wheels/d2/a7/53/9983aef9861f91881984e08f376e3119bdfeeecf55bd36e585
Successfully built img2pdf


In [ ]:
import os
import img2pdf
import PIL #img2pdfと一緒にインストールされたPillowを使います
import natsort

In [ ]:
pdf_FileName = "/content/result.pdf" # 出力するPDFの名前
png_Folder = "/content/result_img/" # 画像フォルダ
extension  = ".png" # 拡張子がPNGのものを対象

with open(pdf_FileName,"wb") as f:
    # 画像フォルダの中にあるPNGファイルを取得し配列に追加、バイナリ形式でファイルに書き込む
    f.write(img2pdf.convert([PIL.Image.open(png_Folder+j).filename for j in natsort.natsorted(os.listdir(png_Folder)) if j.endswith(extension)]))

In [ ]:
#画像フォルダー削除（連続で実施することを想定）
!rm -r /content/result_img
!rm -r /content/target_img
!rm -r /content/source_img

In [ ]:
# https://campkougaku.com/2020/04/15/alignment/
# ズレ修正　これが実装できる日がくるのか・・・。